In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost
import matplotlib.pyplot as plt

def predict(data,thr):
    
    data = data[data['pos'].notna()]
    # print(data)
    data[['pos','neg','neu']] = data[['pos','neg','neu']].pct_change()


    data[['pos','neg','neu']] = data[['pos','neg','neu']].shift(1)
    # data[['pos','neg','neu']] = data[['pos','neg','neu']].rolling(3).mean()

    data['label'] = data['Close'].pct_change()

    print(data['label'].corr(data['pos']))


    data = data[data['label'].notna()]
    data['label'] = data['label'].map(lambda x: 1 if x >=thr else 0)
    # lag
    # data[['pos','neg','neu']] = data[['pos','neg','neu']].shift(1)






    # 
    # df = pd.read_csv('predict_model_train_data.csv')
    df = data
    # print(df)
    features = df[['pos']]
    labels = df['label']
    # 假设数据集X为特征矩阵，y为目标变量
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42,shuffle=False)

    # 定义模型参数
    params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',  # 二分类任务
        'eval_metric': 'logloss',         # 评估指标为logloss，在训练过程中输出
        'eta': 0.001,                       # 学习率，控制每次迭代的步长，默认为0.3
        'max_depth': 3,                   # 决策树最大深度
        'min_child_weight': 1,            # 叶子节点的最小权重
        'gamma': 0.1,                     # 节点分裂时损失函数的下降阈值
        'subsample': 0.8,                 # 每棵树随机采样的比例
        'colsample_bytree': 0.8,          # 每棵树进行特征采样的比例
        'reg_alpha': 0.01,                # L1正则化系数
        'reg_lambda': 0.01,               # L2正则化系数
        'seed': 42                        # 随机种子，保证结果可复现
    }

    # params = {
    #     'booster': 'gbtree',
    #     'objective': 'binary:logistic',  # 二分类任务
    #     'eval_metric': 'logloss',         # 评估指标为logloss，在训练过程中输出
    # }
    # 将数据转换为DMatrix格式，优化XGBoost性能
    dtrain = xgboost.DMatrix(X_train, label=y_train)
    dvalid = xgboost.DMatrix(X_test, label=y_test)



    # 训练模型
    evals_result = {}
    model = xgboost.train(params=params, dtrain=dtrain ,num_boost_round=1000,evals=[(dtrain, 'train'), (dvalid, 'valid')],evals_result=evals_result,verbose_eval=True)
    # 提取损失值
    # print(evals_result['train'])
    train_loss = evals_result['train']['logloss']
    valid_loss = evals_result['valid']['logloss']
    # 绘制损失曲线
    # plt.figure(figsize=(10, 6))
    plt.plot(range(len(train_loss)), train_loss, label='Train Loss')
    plt.plot(range(len(valid_loss)), valid_loss, label='Valid Loss')
    plt.xlabel('Iteration')
    plt.ylabel('logloss')
    plt.title('Training Loss')
    plt.legend()
    plt.show()
    # 将测试集转换为DMatrix格式
    dtest = xgboost.DMatrix(X_test)

    # 在测试集上预测
    y_pred = model.predict(dtest)

    # 将预测结果转换为0-1二分类
    y_pred_binary = [1 if i > 0.5 else 0 for i in y_pred]

    # 计算准确率
    print('accuracy_score:', accuracy_score(y_test, y_pred_binary))
    print('precision_score:', precision_score(y_test, y_pred_binary))
    print('recall_score:', recall_score(y_test, y_pred_binary))
    print('f1_score:', f1_score(y_test, y_pred_binary))

    from xgboost import plot_importance
    from matplotlib import pyplot
    plot_importance(model)
    pyplot.show()



def get_best(join,lag, windows_size):
    join['metrics_return'] = join['metrics_return'].shift(lag)
    # window_size = 7 # 窗口大小
    join['metrics_return'] = join['metrics_return'].rolling(windows_size).mean()

    join = join[join['metrics_return'].notna()]
    t1 = join['metrics_return'].to_list()
    t2 = join['value_return'].to_list()
    import numpy as np
    corr = np.corrcoef(t1, t2)[0, 1]
    return corr

data = pd.read_csv('data/predict_model_data_0730.csv')

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


data['metrics'] =  data['pos'] - data['neg'] 
data['value'] = data['Close']

# data['metrics'] = data['metrics'].shift(1)
data[['metrics']]= scaler.fit_transform(data[['metrics']])
data['metrics_return'] = data['metrics'].pct_change()
data = data[data['metrics_return'] < 9999]
data['value_return'] = data['value'].pct_change()
data = data[data['value_return'] < 9999]


start = '2022-01-01'
end = '2022-12-31'
data['date'] = pd.to_datetime(data['date'])
data = data[(data['date']>= pd.Timestamp(start)) & (data['date'] <= pd.Timestamp(end))]


# corr = get_best(data.copy(), 4,3)
# print(corr)

corr_l = []
for windows in range(30):
    for i in range(30):
        corr = get_best(data.copy(), i+1,windows+1)
        corr_l.append(corr)
        print(windows+1, i+1,corr)
print(max(corr_l))
# data['metrics_return'] = data['metrics_return'].shift(4)
# data['metrics_return'] = data['metrics_return'].rolling(3).mean()
# print(data['metrics_return'].corr(data['value_return']))

# data[['pos','neg','neu']] = data[['pos','neg','neu']].shift(4)
# data[['pos','neg','neu']] = data[['pos','neg','neu']].rolling(3).mean()
# predict(data.copy(), 0)
# predict(data.copy(), -0.05)

1 1 0.043352612851433034
1 2 -0.02977027546241889
1 3 0.0979337613471636
1 4 -0.045471059591785046
1 5 0.027058493135394313
1 6 -0.0005501217867293884
1 7 -0.06187994261944037
1 8 -0.00816774278603794
1 9 0.0493812951598349
1 10 -0.044508561154003534
1 11 0.033564351746512525
1 12 0.07320035495611502
1 13 -0.04028486276673813
1 14 -0.04584304057061991
1 15 0.07457954827811425
1 16 -0.10356311641876835
1 17 0.08152562326682465
1 18 0.0160886090677308
1 19 -0.03775221290079445
1 20 0.014589930365813585
1 21 0.024325324993108684
1 22 -0.11771417225468266
1 23 -0.03560585453381145
1 24 0.05313980027076301
1 25 0.033190504508039634
1 26 -0.0353394822541186
1 27 0.05300564568919706
1 28 -0.03615249263330931
1 29 0.04294314727459109
1 30 0.002752135787096264
2 1 0.010863067828290888
2 2 0.05456183354187564
2 3 0.04206890611412401
2 4 -0.015557531251080833
2 5 0.02121538119135354
2 6 -0.050658437228419256
2 7 -0.05623982649582772
2 8 0.03320260564218458
2 9 0.00393447938472137
2 10 -0.00855481

In [4]:
# # 通过遍历的方式，在不同的学习率的情况下，看哪个学习率的情况下，模型最优
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import StratifiedKFold

# # 设定要调节的 learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3] 

# # 和原代码相比就是在 model 后面加上 grid search 这几行：
# model = xgboost.XGBClassifier()

# learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.5]
# subsample = [0.8, 0.9]
# colsample_bytree = [0.6, 0.8]
# max_depth = [3,5,8]
# min_child_weight = [1,2,4,5]            # 叶子节点的最小权重

# parameters = { 
#     'eta': learning_rate,
#     'subsample': subsample,
#     'colsample_bytree':colsample_bytree,
#     'max_depth': max_depth,
#     'min_child_weight': min_child_weight
#     }


# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
# grid_search = GridSearchCV(model, parameters, scoring="recall", n_jobs=-1, cv=kfold)
# grid_result = grid_search.fit(features, labels)

# # 输出最佳学习率和其对应的分数
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# best_model = grid_result.best_estimator_


# # 打印出每一个学习率对应的分数
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))
